In [1]:
#Importing neccessary libraries
import pandas as pd
import numpy as np
from collections import deque
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import random
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout,BatchNormalization,Flatten

In [2]:
#declare constants

#Timestep is the number of previous days to consider for predicting next days price.
#Essentially Timestep = No. of features for the algorith

timestep = 30

#future_predict is the value to be predicted, eg: value after 1 day, after 2 days,etc.

future_predict = 1

epochs = 10
batch_size = 128

In [3]:
dataset = pd.read_csv("F:/Git_projects/LSTM_time series/RELIANCE.csv")
dataset.head()


,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,1/3/2000,RELIANCE,EQ,233.05,237.50,251.70,237.50,251.70,251.70,249.37,4456424,1.111320e+14,NaN,NaN,NaN
1,1/4/2000,RELIANCE,EQ,251.70,258.40,271.85,251.30,271.85,271.85,263.52,9487878,2.500220e+14,NaN,NaN,NaN
2,1/5/2000,RELIANCE,EQ,271.85,256.65,287.90,256.65,286.75,282.50,274.79,26833684,7.373700e+14,NaN,NaN,NaN
3,1/6/2000,RELIANCE,EQ,282.50,289.00,300.70,289.00,293.50,294.35,295.45,15682286,4.633250e+14,NaN,NaN,NaN
4,1/7/2000,RELIANCE,EQ,294.35,295.00,317.90,293.00,314.50,314.55,308.91,19870977,6.138390e+14,NaN,NaN,NaN


In [15]:
#We will only be predicting "Close" price 
data = dataset.filter(["Close"],axis = 1)

In [18]:
data = data[-30:]

In [19]:
data

,Close
5257,2059.50
5258,2083.25
5259,2067.70
5260,2080.30
5261,2008.10
5262,2023.45
5263,2061.00
5264,2144.35
5265,2085.80
5266,2101.70


In [5]:
def predict(current,future):
    if float(future) > float(current):
        return 1
    else:
        return 0
    
def preprocess(df):
    df.drop("Future",inplace = True,axis = 1)
    df["Close"] = df["Close"].pct_change()
    df.dropna(inplace = True)
    df["Close"] = preprocessing.scale(df["Close"])
    seq_data = []
    prev_days = deque(maxlen = timestep)
    for i in tqdm(df.values):
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == timestep:
            seq_data.append([np.array(prev_days),i[-1]])
        random.shuffle(seq_data)
        X =[]
        y = []
        for seq,target in seq_data:
            X.append(seq)
            y.append(target)
    return np.array(X),np.array(y)

In [6]:
data["Future"] = data["Close"].shift(-future_predict)
data["Target"] = list(map(predict,data["Close"],data["Future"]))
data,data_test = train_test_split(data,test_size = 0.10,shuffle = False)
X_train,y_train = preprocess(data)
X_test,y_test = preprocess(data_test)

100%|██████████| 528/528 [00:00<00:00, 7233.05it/s]


In [7]:
X_train.shape

(4728, 30, 1)

In [8]:
model = Sequential()

model.add(LSTM(128,input_shape = (X_train.shape[1:]),return_sequences = True,activation = "tanh")) 
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(LSTM(128,activation = "tanh",return_sequences = True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128,activation = "tanh"))
model.add(Dropout(0.2))
model.add(BatchNormalization())


model.add(Dense(32,activation = "relu"))
model.add(Dropout(0.2))

model.add(Dense(2,activation = "softmax"))

opt = tf.keras.optimizers.Adam(lr=0.001,decay = 1e-6)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 128)           66560     
_________________________________________________________________
dropout (Dropout)            (None, 30, 128)           0         
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 128)           512       
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 128)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 128)           512       
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               1

In [9]:
model.compile(loss = "sparse_categorical_crossentropy",optimizer= opt ,metrics = "accuracy")


In [10]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = epochs,batch_size = batch_size)

Epoch 1/10
37/37 [==============================] - 13s 193ms/step - loss: 0.8903 - accuracy: 0.4857 - val_loss: 0.6968 - val_accuracy: 0.5030
Epoch 2/10
37/37 [==============================] - 5s 122ms/step - loss: 0.7695 - accuracy: 0.5023 - val_loss: 0.6960 - val_accuracy: 0.5030
Epoch 3/10
37/37 [==============================] - 5s 128ms/step - loss: 0.7430 - accuracy: 0.5024 - val_loss: 0.7018 - val_accuracy: 0.5030
Epoch 4/10
37/37 [==============================] - 5s 133ms/step - loss: 0.7314 - accuracy: 0.5191 - val_loss: 0.7011 - val_accuracy: 0.4890
Epoch 5/10
37/37 [==============================] - 5s 144ms/step - loss: 0.7253 - accuracy: 0.5110 - val_loss: 0.6994 - val_accuracy: 0.4830
Epoch 6/10
37/37 [==============================] - 5s 129ms/step - loss: 0.7119 - accuracy: 0.5211 - val_loss: 0.6991 - val_accuracy: 0.4850
Epoch 7/10
37/37 [==============================] - 6s 153ms/step - loss: 0.7148 - accuracy: 0.5081 - val_loss: 0.7024 - val_accuracy: 0.4749
Epoch

In [11]:
model.save("LSTM_stock")

INFO:tensorflow:Assets written to: LSTM_stock\assets


INFO:tensorflow:Assets written to: LSTM_stock\assets
